# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

592

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [7]:
# API set up
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="

# Lists to hold data
c_id = []
c_name = []
country = []
lat = []
long = []
#weather_date = []
t_max = []
humidity = []
cloudiness = []
w_speed = []

# Create a file to store Open WeatherMap API call info and store as a .csv
owmlog = open('open_weathermap_log.csv', 'w')
owmdata = open('open_weathermap_data.csv', 'w')

# Define header for open_weathermap_data.csv
owmdata.write("c_id,c_name,country,lat,long,weather_date,temp,humidity,cloudiness,w_speed\n")

# Loop through the list of cities
for city in cities:
    query_url = f"{url}{api_key}&q={city}"
    print(query_url)
    response = requests.get(query_url).json()
    
# Try to querry the pertinent data, creating variables with proper formatting
    try:
        cityID = int(response['id'])
        cityName = response['name']
        countryCode = response['sys']['country']
        lat_val = response['coord']['lat']
        lon_val = response['coord']['lon']
        #weatherDate = (datetime.fromtimestamp(response['dt'])).strftime('%m/%d/%y')
        temp_max = response['main']['temp_max']
        humidity_val = response['main']['humidity']
        cloudiness_val = response['clouds']['all']
        windSpeed = response['wind']['speed']
        
        # Append retreived values to lists above using the variables created above
        c_id.append(cityID)
        c_name.append(cityName)
        country.append(countryCode)
        lat.append(lat_val)
        long.append(lon_val)
        #weather_date.append(weatherDate)
        t_max.append(temp_max)
        humidity.append(humidity_val)
        cloudiness.append(cloudiness_val)
        w_speed.append(windSpeed)
       
        # Print cityname & cityID  to mark progress and store it to a print log text file
        print(f"{cityName}:{cityID} was found in the Open Weathermap API\n")
        owmlog.write(f"{cityName}:{cityID} was found in the Open Weathermap API\n")
        
        # Store the data as a .csv file
        owmdata.write(f"{cityID},{cityName},{countryCode},{lat_val},{lon_val},\
            {t_max},{humidity_val},{cloudiness_val},{windSpeed}\n")
        
    # If the data is not in Open WeatherMap API and it creates a Null
    except:
        
        print(f"{city} was not found in the Open Weathermap API\n")
        owmlog.write(f"{city} was not found in the Open Weathermap API\n")
        
        #Add these cities to the lists
        c_id.append(None)
        c_name.append(None)
        country.append(None)
        lat.append(None)
        long.append(None)
        #weather_date.append(None)
        t_max.append(None)
        humidity.append(None)
        cloudiness.append(None)
        w_speed.append(None)
        pass
    
# Close the file handle after completion of the loop
#owmlog.close()
#owmdata.close()

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=puerto ayora
Puerto Ayora:3652764 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=albany
Albany:5106841 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=cherskiy
Cherskiy:2126199 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=kazalinsk
kazalinsk was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=sao joao da barra
São João da Barra:3448903 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=kodiak
Saint Paul Harbor:5866583 was found

Mayo:4362001 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=mys shmidta
mys shmidta was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=pevek
Pevek:2122090 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=udachnyy
Udachny:2014624 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=saleaula
saleaula was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=thompson
Thompson:6165406 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=port alfred
Port Alf

Brae:2654970 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=goderich
Goderich:5962442 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=georgetown
George Town:1735106 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=broome
Broome:5110365 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=castro
Castro:3466704 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=butaritari
Butaritari:2110227 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=half moon 

Cidreira:3466165 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=ishinomaki
Ishinomaki:2112576 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=jumla
Jumla:1283285 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=axim
Esim:2303611 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=henties bay
Henties Bay:3356832 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=vokhma
Vokhma:472887 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=zhangye
Zhangye:


http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=srandakan
Srandakan:1626493 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=batemans bay
Batemans Bay:2176639 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=alexandria
Alexandria:361058 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=san matias
San Matías:3444105 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=clarence town
Clarence Town:2171465 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=sakakah
Sakakah:102527 was found i

...

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=toftir
toftir was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=sao filipe
São Filipe:3374210 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=calbuco
Calbuco:3897334 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=marsala
Marsala:2524245 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=kandiari
Kandiari:1175012 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=linxia
Linxia Chengguanzhen:1803331 was found in the Open Weathermap A

Alice Springs:2077895 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=codrington
Codrington:2171099 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=ust-nera
Ust-Nera:2120048 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=nouadhibou
Nouadhibou:2377457 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=tir pol
tir pol was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=grao mogol
Grão Mogol:3462103 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e

Aktau:610612 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=umzimvubu
umzimvubu was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=abnub
Abnūb:362973 was found in the Open Weathermap API

abnub was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=maceio
Maceió:3395981 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=vega de alatorre
Vega de Alatorre:3514843 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=ambilobe
Ambilobe:1082243 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&A

gorno-chuyskiy was not found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=northam
Northam:2641434 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=hobyo
Hobyo:57000 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=prainha
Prainha:3391287 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=lavrentiya
Lavrentiya:4031637 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=dauphin
Dauphin:5935341 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=turtkul
turtkul

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=la peca
La Peca:3695675 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=caronport
Caronport:5917430 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=buchanan
Buchanan:2278158 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=jiuquan
Jiuquan:1279945 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=santa ines
Santa Inês:3389609 was found in the Open Weathermap API

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=809d895ef321b812b0cf144f02b8b8e5&q=sarangani
Sarangani:1687186 was found in the Open Weathermap

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [10]:
cities_df.to_csv("open_weathermap_data.csv")

NameError: name 'cities_df' is not defined

In [9]:
cities_df = pd.DataFrame(city_list)

cities_df.head()

NameError: name 'city_list' is not defined

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["temp_max"], marker = "^")
plt.title("Latitude vs. Max Temperature")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature")

plt.savefig("Pics/Lat_v_max_temp.png")
plt.show()

print("Conclusion")

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["humidity"], marker = "^")
plt.title("Latitude vs. Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity")

plt.savefig("Pics/Lat_v_Humidity.png")
plt.show()

print("Conclusion")

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["cloudiness"], marker = "^")
plt.title("Latitude vs. Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")

plt.savefig("Pics/Lat_v_Cloudiness.png")
plt.show()

print("Conclusion")

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(cities_df["lat"], cities_df["w_speed"], marker = "^")
plt.title("Latitude vs. Wind Speed")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")

plt.savefig("Pics/Lat_v_w_speed.png")
plt.show()

print("Conclusion")

## Linear Regression

In [ ]:
#Divide the cities into Northern and Southern Hemisphere DataFrames
northern_df = cities_df.loc[cities_df["lat"].astype(float) > 0]
southern_df = cities_df.loc[cities_df["lat"].astype(float) < 0]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Run Linear Regression Analysis
x_values = northern_df["lat"]
y_values = northern_df["temp_max"]

# Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

# Plot the regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

# Title & Axis labels
plt.title("Northern Hemisphere - Temperature (F) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Lat_v_Max_temp.png")
plt.show()

print("conclusion")

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = southern_df["lat"]
y_values = southern_df["temp_max"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Temperature (F) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Max Temp (F)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Lat_v_temp_max.png")
plt.show()

print("conclusion")

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = northern_df["lat"]
y_values = northern_df["humidity"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Hum_vs_Lat.png")
plt.show()

print("conclusion")

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = southern_df["lat"]
y_values = southern_df["humidity"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Southern Hemisphere - Humidity (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/South_Hem_Hum_vs_Lat.png")
plt.show()

print("conclusion")

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Analysis
x_values = northern_df["lat"]
y_values = northern_df["clouds"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Cloudiness (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_Cloudi_vs_Lat.png")
plt.show()

print("Conclusion")

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:

#Run Linear Regression Data
x_values = southern_df["lat"]
y_values = southern_df["clouds"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Southern Hemisphere - Cloudiness (%) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")

# Display the analysis results
print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/South_Hem_Cloudi_vs_Lat.png")
plt.show()

print("Conclusion")

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Data
x_values = northern_df["lat"]
y_values = northern_df["wind"]

#Plot scatter plot
plt.scatter(x_values,y_values)

# Regression line info
(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))

#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/North_Hem_W_Speed_vs_Lat.png")
plt.show()

print("Conclusion.")

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
#Run Linear Regression Data
x_values = southern_df["lat"]
y_values = southern_df["wind"]

#Plot scatter plot
plt.scatter(x_values,y_values)


(slope, intercept, rvalue, pbalue, stderr) = linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y= " + str(round(slope,2)) + "x+ " + str(round(intercept,2))



#Plot regression line
plt.plot(x_values,regress_values,"r--")
plt.annotate(line_eq,(0,50), fontsize = 6, color = "green")

#Title & Axis labels
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude (mph)")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")

print(f"Regression line equation is: {line_eq}")
print(f"The r-squared is: {rvalue**2}")

plt.savefig("Pics/South_Hem_W_Speed_vs_Lat.png")
plt.show()

print("Conclusion")